# Sentiment Analysis Using RNN

In [1]:
import numpy as np
from string import punctuation
from collections import Counter, OrderedDict
import itertools

import torch 
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn

import nltk
from nltk.corpus import stopwords

In [103]:
with open('deep-learning-v2-pytorch/sentiment-analysis-network/reviews.txt', 'r') as f:
    reviews = f.read()
with open('deep-learning-v2-pytorch/sentiment-analysis-network/labels.txt', 'r') as f:
    labels = f.read()

In [104]:
# this shows that the files were imported character by character
labels[:100]

'positive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nnegative\npositive\nn'

In [105]:
sample = reviews[:3000]

In [97]:
# remove punctuation
s = ''.join(ch for ch in sample.lower() if ch not in punctuation)


In [108]:
test = clean_text(reviews)